#Dolar Exchange Project<br/>


In [0]:
import pyspark
import pandas as pd
from datetime import datetime,timedelta,date
import requests


##1-Connection to ADLS

In [0]:
#Date paramters
todays_date = (date.today()).strftime("%m-%d-%Y")
yesterday = (date.today() - timedelta(days=1)).strftime("%m-%d-%Y") #Today's date -1 day

In [0]:
#Create some widgets to safely store the information from our data lake
dbutils.widgets.text(name="Storage_Account",defaultValue="")
dbutils.widgets.text(name="Storage_Key",defaultValue="")
dbutils.widgets.text(name="Storage_Scope",defaultValue="")

In [0]:
#Acess the widgets to get our key and scope
scope = dbutils.widgets.get("Storage_Scope")
key = dbutils.widgets.get("Storage_Key")
storageAccount = dbutils.widgets.get("Storage_Account")
containerName = "ingrid-sollim"
mountpoint = "/mnt/ingrid-sollim/"
storageEndpoint = f"wasbs://{containerName}@{storageAccount}.blob.core.windows.net" 
storageKey = dbutils.secrets.get(scope=scope,key=key)
storageConn = f"fs.azure.account.key.{storageAccount}.blob.core.windows.net"

try:
    if not any(mount.mountPoint==mountpoint for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
        source = storageEndpoint,
        mount_point = mountpoint,
        extra_configs = {storageConn:storageKey}
    )
        print(f"{mountpoint} has been mounted")
    else:
        print(f"Mount point '{mountpoint}' is already mounted.")
except Exception as e:
    raise e  # Re-raise the exception if mounting fails

Mount point '/mnt/ingrid-sollim/' is already mounted.


In [0]:
path = mountpoint+f"dolar/{todays_date}/{todays_date}.parquet"

In [0]:
df = spark.read.parquet(path)
display(df.limit(10))

cotacaoCompra,cotacaoVenda,dataHoraCotacao
"5,166","5,1666",2023-10-09 13:22:11.067


##2-Change Data types

In [0]:
df.printSchema()

root
 |-- cotacaoCompra: string (nullable = true)
 |-- cotacaoVenda: string (nullable = true)
 |-- dataHoraCotacao: string (nullable = true)



In [0]:
from pyspark.sql.functions import col,regexp_replace,to_timestamp

In [0]:
#Replace "," for "."
replace_df = df.withColumn("cotacaoCompra",regexp_replace(col("cotacaoCompra"),",",".").cast("double")).withColumn("cotacaoVenda",regexp_replace(col("cotacaoVenda"),",",".").cast("double"))
display(replace_df.limit(10))

cotacaoCompra,cotacaoVenda,dataHoraCotacao
5.166,5.1666,2023-10-09 13:22:11.067


In [0]:
#Convert the data types
new_df = replace_df.withColumn("dataHoraCotacao",to_timestamp(col("dataHoraCotacao"),"yyyy-MM-dd HH:mm:ss.SSS"))
display(new_df.limit(10))

cotacaoCompra,cotacaoVenda,dataHoraCotacao
5.166,5.1666,2023-10-09T13:22:11.067+0000


In [0]:
new_df.printSchema()

root
 |-- cotacaoCompra: double (nullable = true)
 |-- cotacaoVenda: double (nullable = true)
 |-- dataHoraCotacao: timestamp (nullable = true)



##3-Ingest into the database

In [0]:
#Create some widgets to safely store the information from our database connection
dbutils.widgets.text(name="db_scope",defaultValue="")
dbutils.widgets.text(name="db_username_key",defaultValue="")
dbutils.widgets.text(name="db_pass_key",defaultValue="")

###Create a connection to the database

In [0]:
#Get the secrets
username = dbutils.secrets.get(scope=dbutils.widgets.get("db_scope"),key=dbutils.widgets.get("db_user_name"))
password = dbutils.secrets.get(scope=dbutils.widgets.get("db_scope"),key=dbutils.widgets.get("db_pass_key"))

jdbcUsername = username
jdbcPassword = password
jdbcHostname = "srv-fab-projetos.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "db-fab-projetos"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

table_name = 'sollim.dolar'

###Ingest into the database

In [0]:
new_df.write.format("jdbc")\
    .option("url",jdbcUrl)\
    .option("dbtable",table_name)\
    .mode("append")\
    .options(**connectionProperties) \
    .save()

In [0]:
dbutils.fs.unmount(mountpoint)

/mnt/ingrid-sollim/ has been unmounted.
Out[36]: True